In [69]:
from pymystem3 import Mystem
import nltk
import pandas as pd

import re
import json
from os import listdir
from os import path

In [70]:
articles_dir = "parsed_articles"
article_files = [f for f in listdir(articles_dir) if path.isfile(path.join(articles_dir, f))]

articles = dict()

for article_file in article_files:
    article_path = path.join(articles_dir, article_file)
    # print(article_path)
    with open(article_path, 'r', encoding="UTF-8") as f:
        article = f.read()
        article_json = json.loads(article)
        articles[article_json["article_id"]] = article_json
        # print(article_json)

In [71]:
from nltk.corpus import stopwords
m = Mystem()
# ntlk.downlo
stopwords = set(stopwords.words('russian'))

def prepare_text(text):
    prepared_text = text
    prepared_text = re.sub(r'\s', ' ', prepared_text)
    prepared_text = re.sub(r'[^\w ]', '', prepared_text)
    prepared_text = prepared_text.lower()
    prepared_text = " ".join(filter(lambda x: x not in stopwords and len(x) < 30, prepared_text.split(" ")))
    prepared_text = ''.join(m.lemmatize(prepared_text))
    prepared_text = prepared_text.strip()
    prepared_text = prepared_text[:8000]
    
    return prepared_text

def prepare_tags(tags):
    prepared_tags = tags
    prepared_tags = map(lambda tag: tag.lower(), prepared_tags)
    prepared_tags = map(lambda tag: tag.replace("-", " "), prepared_tags)
    
    return list(prepared_tags)

In [72]:
df = pd.DataFrame(articles.values())

In [73]:
df['content'] = df['content'].apply(prepare_text)
df['tags'] = df['tags'].apply(prepare_tags)

In [74]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['tags'])
len(mlb.classes_)

3648

In [75]:
df["tags_bins"] = df["tags"].apply(lambda x: mlb.transform([x])[0])
df

,article_id,article_name,content,tags,tags_bins
0,729250,Rust Foundation хочет запретить менять цвета л...,предупреждение низко это запаздывать первоапре...,"[rust, копирайт]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,729060,Самый детальный разбор закона об электронных п...,новый закон электронный повестка потенциально ...,"[законодательство, мобилизация, повестка, госу...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,728856,Идеальное компьютерное кресло — миф или реальн...,привет хабр продолжать разбираться существоват...,"[рабочее место, рабочее место программиста, кр...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,728644,Стилмэнинг и идеологический тест Тьюринга,мысль начинаться наталкиваться препятствие лев...,"[стилменинг, идеологический тест тьюринга, сол...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,728748,RMT в World of Warcraft: почему Blizzard терпи...,промпт game world of warcraft fantasy doge mon...,"[world of warcraft, blizzard, kandinsky art, н...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
994,726292,Как работает веб-браузер (с картинками),иллюстрация предоставлять growtika браузер ст...,"[браузеры, dns поиск, tcp, tls, токенизация, j...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
995,726306,Ближайшие бесплатные мероприятия по разработке...,1 апрель начало 1100 мск суббота онлайн t...,"[митап, конференция, mobile, документация, rab...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
996,726316,Bash скрипты,работа командный строка linux shell скрипт на...,"[bash, bash скрипт, bash scripting, shell]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
997,726308,ADD – добавим гибкости,такой add короче add английский аббревиатура...,"[гибкая разработка по, методология разработки,...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.3, random_state=120)

In [19]:
# for article in df['content']:
#     print(len(article))

# import seaborn as sns
# sns.set_theme()
# sns.displot(df['content'], x="x name")

# articles_lens = map(lambda x: len(x), df['content'])

# import matplotlib.pyplot as plt
# import numpy as np
# counts, bins = np.histogram(articles_lens)
# plt.stairs(counts, bins)
# list(articles_lens)

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tdidf_vectorizer = TfidfVectorizer(decode_error='ignore')
X_train_tdidf_vectorized = tdidf_vectorizer.fit_transform(X_train['content'])
X_test_tdidf_vectorized = tdidf_vectorizer.transform(X_test['content'])

In [78]:
# classifier v1
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

#обучаем подель SVM

import warnings
warnings.filterwarnings('ignore')
print("warnings will be ignored")

model = OneVsRestClassifier(LinearSVC(random_state = 50))
model.fit(X_train_tdidf_vectorized, y_train)

warnings will be ignored


OneVsRestClassifier(estimator=LinearSVC(random_state=50))

In [79]:
predictions = model.predict(X_test_tdidf_vectorized)
pred_tags = mlb.inverse_transform(predictions)
list(zip(X_test['article_id'], pred_tags))
# for tags, pred_tags in zip(X_test['tags'], pred_tags):
    # print(f"tags: {tags} \nclassified: {pred_tags}\n\n")

[('725200', ()),
 ('727478', ()),
 ('727810', ()),
 ('726868', ()),
 ('726228', ()),
 ('726502', ()),
 ('726144', ()),
 ('725650', ()),
 ('727184', ('chatgpt',)),
 ('727094', ()),
 ('728414', ()),
 ('727738', ()),
 ('727010', ()),
 ('728184', ()),
 ('727830', ()),
 ('728100', ('chatgpt',)),
 ('729206', ()),
 ('728172', ()),
 ('726168', ()),
 ('728434', ()),
 ('728984', ()),
 ('728084', ()),
 ('726662', ()),
 ('725578', ()),
 ('726174', ()),
 ('725870', ()),
 ('729016', ()),
 ('726098', ()),
 ('725752', ()),
 ('725648', ()),
 ('727884', ()),
 ('727938', ()),
 ('727400', ()),
 ('726514', ('менеджмент', 'производительность труда')),
 ('728772', ()),
 ('726400', ()),
 ('726722', ()),
 ('729172', ()),
 ('727592', ()),
 ('727780', ()),
 ('725958', ()),
 ('726092', ()),
 ('726236', ()),
 ('727498', ('kafka',)),
 ('728720', ()),
 ('723454', ()),
 ('726106', ()),
 ('727312', ()),
 ('725750', ()),
 ('726852', ()),
 ('727826', ()),
 ('725884', ()),
 ('725800', ()),
 ('726788', ()),
 ('727718', ()

In [80]:
model.score(X_test_tdidf_vectorized, y_test)

0.0033333333333333335

In [49]:
for i in range(len(df['content'])):
    with open(f"test_tmp/article_{i}_content", 'w') as f:
        f.write(df['content'][i])

In [8]:
import/text.train_unsupervised('test_tmp/article_0_content')

Read 0M words
Number of words:  5
Number of labels: 0
Progress: 100.0% words/sec/thread:    2315 lr:  0.000000 avg.loss:  4.117939 ETA:   0h 0m 0s


In [14]:
# for i in range(len(df['content'])):
#     model = fasttext.train_unsupervised(f'test_tmp/article_{i}_content')
#     print(df['article_id'][i], df['tags'][i], model.words)

In [15]:
model.words

['робот',
 'процесс',
 'uipath',
 'rpa',
 'проект',
 'pix',
 'работа',
 'находить',
 'библиотека',
 'нужно',
 'который',
 'элемент',
 'это',
 'задача',
 'год',
 'далеко',
 'работать',
 'решение',
 'делать',
 'hoff',
 'запускать',
 'активность',
 'разработка',
 '2022']

In [54]:
# df['article_id'].index("723214")
#"723214"
a = df.loc[df['article_id'] == "723214"]
a

,article_id,article_name,content,tags
642,723214,Как создавать SBOM в Java с помощью Maven и Gr...,При создании приложений на Java мы сильно зави...,"[devops *, безопасность, security]"


In [68]:
prepare_text(a['content'].values[0])

'создание приложение java сильно зависеть внешний библиотека фреймворков каждый импортировать пакет java скоро также зависеть больший количество библиотека это означать количество javaпакетов включать ваш приложение часто прозрачно вложенный транзитивный зависимость создавать проблема заключаться разработчик вероятно знать библиотека который самый дело использовать недавно обсуждать почему должный тщательно поддерживать наш зависимость статья   хороший практика управление зависимость java  рассказывать доступный вариант инструмент настройка стратегия управление зависимость передавать свой javaприложение клиент узнавать какой зависимость включать важный мочь проверять уязвимый зависимость проблема безопасность ответ являться  спецификация программный обеспечение  sbom software bill of materials такой sbom спецификация  программный обеспечение  часто сокращенно называть sbom представлять себя список программный компонент использовать приложение sbom состоять сторонний библиотека открытый